# FILTRADO DE DATOS

In [1]:
import pandas as pd
import numpy as np
import os

Eliminamos Departamentos que no pertenecen al país. Además creamos una copia del Dataftrame

In [2]:
# Cargar el archivo CSV (ajusta el nombre y ruta del archivo)
datos = pd.read_csv('Delitos Colombia/Datos2010-2015.csv')

# Convertir de string a datetime con formato original
datos['FECHA_HECHO'] = pd.to_datetime(datos['FECHA_HECHO'], format='%d/%m/%Y')

# Luego convertirlo al formato YYYY-MM-DD como string
datos['FECHA_HECHO'] = datos['FECHA_HECHO'].dt.strftime('%Y-%m-%d')

# Connvierte la columna 'FECHA_HECHO' a formato datatime
datos['FECHA_HECHO'] = pd.to_datetime(datos['FECHA_HECHO'], format="%Y-%m-%d")

# Eliminar departamentos que no pertenecen a Colombia
departamentos_invalidos = ['MADRID', 'QUITO', 'NEW YORK', 'NEW JERSEY', 'FLORIDA']
datos = datos[~datos['DEPARTAMENTO'].str.upper().isin(departamentos_invalidos)]

# Eliminar '(CT)' de los nombres de municipios
datos['MUNICIPIO'] = datos['MUNICIPIO'].str.replace(r'\s*\(CT\)', '', regex=True)

# Convierte todas las columnas de cadena a mayúsculas
str_cols = datos.select_dtypes(include=['object']).columns
datos[str_cols] = datos[str_cols].apply(lambda x: x.str.upper())

# Selecciona las columnas numericas
num_cols = datos.select_dtypes(include=['float64', 'int64']).columns

# Convierte las columnas numericas a tipo entero
datos[num_cols] = datos[num_cols].astype(int)

# Se crea una copia del dataframe para evitar modificar el original
copia = datos.copy()

# Se corrige el nombre de algunos departamentos
copia['DEPARTAMENTO'] = copia['DEPARTAMENTO'].replace({
    'VALLE': 'VALLE DEL CAUCA',
    'GUAJIRA': 'LA GUAJIRA'
})

# Eliminar la columna innecesaria 'CODIGO_DANE'
if 'CODIGO_DANE' in copia.columns:
    copia = copia.drop(columns=['CODIGO_DANE'])


C:\Users\monte\AppData\Local\Temp\ipykernel_2556\62448304.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  datos = pd.read_csv('Delitos Colombia/Datos2010-2015.csv')


Función que convierte nombres de municipios a sus formas oficiales o completas usando un diccionario de correspondencia. Si un nombre está en el diccionario, se reemplaza; si no, se mantiene igual

In [3]:
def convert_names(original_names):
    # Diccionario de correspondencia entre los nombres
    dictionary  = {
        'CALI': 'SANTIAGO DE CALI',
        'LÓPEZ': 'LÓPEZ DE MICAY',
        'EL PAUJIL': 'EL PAUJÍL',
        'MAGÜI': 'MAGÜÍ',
        'CARTAGENA': 'CARTAGENA DE INDIAS',
        'SOTARA': 'SOTARÁ PAISPAMBA',
        'GACHALA': 'GACHALÁ',
        'PURÍSIMA': 'PURÍSIMA DE LA CONCEPCIÓN',
        'ANCUYÁ': 'ANCUYA',
        'CERRO SAN ANTONIO': 'CERRO DE SAN ANTONIO',
        'MARIQUITA': 'SAN SEBASTIÁN DE MARIQUITA',
        'SANTAFÉ DE ANTIOQUIA': 'SANTA FÉ DE ANTIOQUIA',
        'ITAGUI': 'ITAGÜÍ',
        'CACHIRÁ': 'CÁCHIRA',
        'ANZA': 'ANZÁ',
        'CARMEN DEL DARIEN': 'CARMEN DEL DARIÉN',
        'CUASPUD': 'CUASPUD CARLOSAMA',
        'COLOSO': 'COLOSÓ',
        'SAN ANDRÉS SOTAVENTO': 'SAN ANDRÉS DE SOTAVENTO',
        'SAN JUAN DE RÍO SECO': 'SAN JUAN DE RIOSECO',
        'GÜICÁN': 'GÜICÁN DE LA SIERRA',
        'SABANAS DE SAN ANGEL': 'SABANAS DE SAN ÁNGEL',
        'CAQUEZA': 'CÁQUEZA',
        'MOMPÓS': 'SANTA CRUZ DE MOMPOX',
        'CHAMEZA': 'CHÁMEZA',
        'CARURU': 'CARURÚ',
        'BELÉN DE LOS ANDAQUIES': 'BELÉN DE LOS ANDAQUÍES',
        'CONSACA': 'CONSACÁ',
        'CÚCUTA': 'SAN JOSÉ DE CÚCUTA',
        'GAMBITA': 'GÁMBITA',
        'ENTRERRIOS': 'ENTRERRÍOS',
        'TORIBIO': 'TORIBÍO',
        'SAN ANDRES DE TUMACO': 'SAN ANDRÉS DE TUMACO',
        'RÍO IRO': 'RÍO IRÓ',
        'EL PIÑON': 'EL PIÑÓN',
        'SONSON': 'SONSÓN',
        'MACHETA': 'MACHETÁ',
        'CALARCA': 'CALARCÁ',
        'TURBANÁ': 'TURBANA',
        'UMBITA': 'ÚMBITA',
        'GUAYABAL DE SIQUIMA': 'GUAYABAL DE SÍQUIMA',
        'GAMEZA': 'GÁMEZA',
        'IQUIRA': 'ÍQUIRA',
        'PIENDAMÓ': 'PIENDAMÓ - TUNÍA',
        'ABREGO': 'ÁBREGO',
        'SAN VICENTE': 'SAN VICENTE FERRER',
        'VILLA DE SAN DIEGO DE UBATE': 'VILLA DE SAN DIEGO DE UBATÉ',
        'DON MATÍAS': 'DONMATÍAS',
        'FOMEQUE': 'FÓMEQUE'
    }

    # Convertir los nombres utilizando el diccionario, o mantener el nombre original si no se encuentra
    converted_names = [dictionary.get(name, name) for name in original_names]
    
    return converted_names


# Aplicar la función a la columna 'MUNICIPIO' del DataFrame clean_data
copia['MUNICIPIO'] = convert_names (copia['MUNICIPIO'])

Separa a BOGOTÁ del conjunto de datos de CUNDINAMARCA y lo convierte en un departamento independiente

In [4]:
#Creamos un nuevo datasets, con los datos ajustados a la geolocalizacion
es_bogota = copia['MUNICIPIO'] == "BOGOTÁ D.C."
bogota_datos = copia[es_bogota].copy()
bogota_datos['DEPARTAMENTO'] = "BOGOTÁ D.C."
copia.loc[es_bogota, 'DEPARTAMENTO'] = "BOGOTÁ D.C."

Se crea una función con el fin de reemplazar valores en una columna del conjunto de datos cuando su frecuencia es menor a un determinado percentil.

In [5]:
import numpy as np

def replace_by_percentile(data, column, prctile, replacement_value):

    # Calcular el percentil de las frecuencias
    frequencia = data[column].value_counts()
    per = np.percentile(frequencia, prctile)
    
    # Reemplazar los valores en las filas donde la frecuencia es menor al percentil dado
    copia.loc[copia[column].map(frequencia) < per, column] = replacement_value
    
    return copia

DelFil = replace_by_percentile(copia, "DESCRIPCION_CONDUCTA_CAPTURA", 88, "OTROS")

# ALMACENAMIENTO DE LOS DATOS

Crea un dataset con los datos limpios a nivel nacional

In [6]:
DelFil.to_csv('Delitos Colombia/Datos Limpios.csv', index=False)

Crea una nueva carpeta 'Delitos po Departamentos', en la cual se guarda un csv para cada departamento

In [7]:
def csv_by_filter(data, filter, folder, out = True):
    
    # Especifica la carpeta donde guardar los archivos CSV
    os.makedirs(folder, exist_ok=True)  # Crea la carpeta si no existe

    # Filtra y guarda un CSV para cada departamento
    for filter, loop_data in data.groupby(filter):
        # Genera el nombre del archivo para cada departamento
        output_file = os.path.join(folder, f"{filter}.csv")
        if (out):
            print(f"Guardando {output_file}... con {len(loop_data)} filas")
        # Guarda el DataFrame filtrado como CSV
        loop_data.to_csv(output_file, index=False)
    if (out):
        print("Archivos CSV generados en la carpeta:", folder)

# Filtra y guarda un CSV para cada departamento
csv_by_filter(DelFil, "DEPARTAMENTO", 'Delitos por Departamento', out = False)